# Classifier LLM test

In [37]:
import pandas as pd
import time

### Pandas dataset generation

In [ ]:
dataset = open("classifier_val_dataset.txt","r")
text = []
label = []
for line in dataset.readlines():
    if line == "\n":
        continue
    line = line.split(":")
    line = [x.strip(" \n") for x in line]
    text.append(line[0])
    label.append(line[1])
dataset = pd.DataFrame({"text": text, "label": label},index=[i+1 for i in range(len(text))])
dataset

,text,label
1,Predict the trend of working_time_avg for Lase...,predictions
2,Predict the future working_time_max for Assemb...,predictions
3,Predict the future offline_time_min for Assemb...,predictions
4,Predict the future offline_time_sum for Riveti...,predictions
5,Can you predict idle_time_avg for Laser Weldin...,predictions
...,...,...
68,Find a dashboard suitable to give an overview ...,dashboard
69,Generate a new random dashboard,dashboard
70,I need to monitor the productivity of the work...,dashboard
71,I need to monitor the energy consumption of my...,dashboard


### Classification test

The notebook starts with the communication initializzation to _Gemini_1.5_flash_ API.<br>
The prompt classification is made using gemini with few shot learning, using an example for each classification label.

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from tqdm import tqdm
import os
import sys

current_dir = %pwd
parent_dir = os.path.abspath(os.path.join(current_dir, '../'))
sys.path.append(parent_dir)

load_dotenv(dotenv_path=parent_dir + "\RAG\.env")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# few shots examples
esempi = [
    {"text": "Predict for the next month the cost_working_avg for Large Capacity Cutting Machine 2 based on last three months data", "label": "predictions"},
    {"text": "Generate a new kpi named machine_total_consumption which use some consumption kpis to be calculated", "label": "new_kpi"},
    {"text": "Compute the Maintenance Cost for the Riveting Machine 1 for yesterday", "label": "kpi_calc"},
    {"text": "Can describe cost_working_avg?", "label": "kb_q"},
    {"text": "Make a report about bad_cycles_min for Laser Welding Machine 1 with respect to last week", "label": "report"},
    {"text": "Create a dashboard to compare perfomances for different type of machines", "label": "dashboard"},
]

# Few shot prompt creation
esempio_template = PromptTemplate(
    input_variables=["text", "label"],
    template="Text: {text}\nLabel: {label}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=esempi,
    example_prompt=esempio_template,
    suffix="Classify with one of the labels ['predictions', 'new_kpi', 'report', 'kb_q', 'dashboard','kpi_calc'] the following prompt:\nText: {text_input}\nLabel:",
    input_variables=["text_input"]
)


Now we can start testing few shot learning on the dataset to fine-tune prompts

In [ ]:
true_pos=0
mismatch = []
total = dataset.shape[0]

with tqdm(total=total) as bar:
    for (text,golden_label) in zip(dataset["text"],dataset["label"]):
        local_time = time.time()
        prompt = few_shot_prompt.format(text_input=text)
        out = llm.invoke(prompt)
        out=out.content.strip("\n")
        if out == golden_label:
            true_pos+=1
        else:
            mismatch.append(f"text = {text}\nTrue label = {golden_label}, predicted = {out}")
        bar.update(1)
        # Request rate must be limited to 4 requests per second (free API limit)
        time.sleep(4 - (time.time() - local_time) + 0.01)
        
        


print(f"Accuracy = {true_pos/total}")
print(f"There have been {len(mismatch)} wrong predictions:")
for i in mismatch:
    print(mismatch)

100%|██████████| 72/72 [04:55<00:00,  4.10s/it]

Accuracy = 1.0
There have been 0 wrong predictions:
